# Autologin to ICICI Direct API using Selenium

In [ ]:
!pip install -U selenium

In [1]:
from selenium import webdriver
import pandas as pd
from breeze_connect import BreezeConnect
import time
import os
from pyotp import TOTP
import urllib

api_key = "ie3wytccnxhinew3" <font color='gray'>- Create App screen ICICI Direct API home page</font><br>
key_secret = "j38hb91ifln6gygbzaw2evlcof8dfor" <font color='gray'> - Create App screen </font><br>
userID = "XY6867" <font color='gray'>- Your UserID</font><br>
pwd = "pass@1234" <font color='gray'>- Your ICICI Direct password</font><br>
totp_key = "GRSCPUZ5CYJAGBLZ5HSS72WNGZAZP" <font color='gray'>ICICI Direct Trading Account ->Settings -> Enable Login Authenticator</font><br>

Create security.txt file in the same directory with Login Info

In [2]:
file = open("security.txt", "r")
keys = file.read().split()  # Get a List of keys
print(keys[2])

SATSIN47


In [3]:
api_key = keys[0]
key_secret = keys[1]
userID = keys[2]
pwd = keys[3]
totp_key = keys[4]

In [4]:
breeze = BreezeConnect(api_key=api_key)

In [5]:
browser = webdriver.Chrome()
browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
browser.implicitly_wait(5)

The chromedriver version (119.0.6045.105) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (121.0.6167.160); currently, chromedriver 121.0.6167.85 is recommended for chrome 121.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 119
Current browser version is 121.0.6167.160 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x0000000104a7e004 chromedriver + 4169732
1   chromedriver                        0x0000000104a75ff8 chromedriver + 4136952
2   chromedriver                        0x00000001046cb500 chromedriver + 292096
3   chromedriver                        0x000000010470195c chromedriver + 514396
4   chromedriver                        0x0000000104700b98 chromedriver + 510872
5   chromedriver                        0x00000001046fc244 chromedriver + 492100
6   chromedriver                        0x00000001046f95b8 chromedriver + 480696
7   chromedriver                        0x000000010473b754 chromedriver + 751444
8   chromedriver                        0x00000001047045e8 chromedriver + 525800
9   chromedriver 

<i> Provide username & password</i>

In [ ]:
username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
password = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input') 

username.send_keys(userID)
password.send_keys(pwd)

#Checkbox
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
# Click Login Button
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
time.sleep(2)

<i>Provide OTP programatically</i>

In [ ]:
pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')
totp = TOTP(totp_key)
token = totp.now()
pin.send_keys(token)

<i>Click button and Get Token</i>

In [ ]:
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
time.sleep(2)
temp_token=browser.current_url.split('apisession=')[1][:8]
# Save in Database or text File
print('temp_token', temp_token)

<i>Generate Session</i>

In [ ]:
breeze.generate_session(api_secret=key_secret,session_token=temp_token)

Test

In [ ]:
breeze.ws_connect()

In [ ]:
def download_icici(sdate, edate, symbol):
    data = breeze.get_historical_data(interval="5minute", from_date= sdate + "T09:15:00.000Z",
                            to_date= edate + "T15:30:00.000Z", stock_code=symbol, exchange_code="NSE", product_type="cash")
    put_data = pd.DataFrame(data["Success"])
    if put_data.empty:
        print('Data not found:', adate)
        df = pd.DataFrame()
        return df
    else:
        df = put_data[['datetime', 'open', 'high', 'low', 'close']]
        df.set_index('datetime', inplace=True)
    return df

In [ ]:
sdate = '2023-01-27'
edate = '2023-01-30'
df = download_icici(sdate, edate,'TCS')

In [ ]:
df